In [ ]:
import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')  # Use TkAgg for interactivity; switch to 'Qt5Agg' if needed
import matplotlib.pyplot as plt
from skimage.io import imread as tiff_imread
from skimage.filters import threshold_otsu
from skimage.morphology import binary_dilation
from skimage.segmentation import watershed
from skimage.measure import label, regionprops
from scipy.ndimage import distance_transform_edt, binary_fill_holes  # Use scipy for binary_fill_holes
import pandas as pd
import skimage
import scipy
from skimage.filters import gaussian, threshold_otsu
from skimage.morphology import binary_opening, disk
from skimage.feature import peak_local_max
import numpy as np



In [ ]:
# Define scale from ImageJ dialog
PIXELS_PER_UM = 18.2044  # 18.2044 pixels = 1 μm
UM_PER_PIXEL = 1 / PIXELS_PER_UM  # ≈ 0.05493 μm/pixel
AREA_UM2_PER_PIXEL2 = UM_PER_PIXEL ** 2  # ≈ 0.003017 μm²/pixel²

In [ ]:
# Define directory and list .lsm files
directory = r'C:\Users\Raymond\OneDrive\Desktop\progress report\confocol\20241209\rh1lacz-atg5'
lsm_files = [f for f in os.listdir(directory) if f.endswith('.lsm')]
all_results = []
all_files_summary = []

In [ ]:
# Segment 1: Load and Validate Image
def load_image(file_path):
    print(f"Loading file: {file_path}")
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} does not exist.")
        return None
    try:
        image = tiff_imread(file_path)
        print("Image shape:", image.shape)
        print("Image min/max:", image.min(), image.max())
        # Check if image is in (height, width, channels) format
        if image.shape[-1] in [3, 4]:  # Likely (height, width, channels)
            image = np.transpose(image, (2, 0, 1))  # Convert to (channels, height, width)
            print("Transposed image shape:", image.shape)
        return image
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

In [ ]:
# Segment 2: Separate Channels
def separate_channels(image):
    if image is None or image.ndim != 3:
        print("Error: Invalid image data for channel separation.")
        return None, None
    # Assuming image is now (channels, height, width)
    purple_channel = image[2]  # Channel 2 (purple, nuclear lamin)
    green_channel = image[0]   # Channel 0 (green, signal to quantify)
    print("Purple channel shape:", purple_channel.shape)
    print("Purple channel min/max:", purple_channel.min(), purple_channel.max())
    print("Green channel shape:", green_channel.shape)
    print("Green channel min/max:", green_channel.min(), green_channel.max())
    return purple_channel, green_channel

In [ ]:
def quantify_nuclei(labeled_nuclei, green_channel):
    if labeled_nuclei is None or green_channel is None:
        print("Error: Invalid data for quantification.")
        return [], []
    initial_results = []
    min_nucleus_area = 2000
    max_nucleus_area = 10000
    valid_nuclei_labels = []
    for region in regionprops(labeled_nuclei, intensity_image=green_channel):
        if min_nucleus_area <= region.area <= max_nucleus_area:
            nucleus_id = region.label
            area_pixels = region.area
            area_um2 = area_pixels * AREA_UM2_PER_PIXEL2
            mean_intensity = region.mean_intensity
            raw_intden = np.sum(region.intensity_image[region.image])
            initial_results.append({
                'Nucleus_ID': nucleus_id,
                'Area_pixels': area_pixels,
                'Area_um2': area_um2,
                'Mean_Intensity': mean_intensity,
                'RawIntDen': raw_intden
            })
            valid_nuclei_labels.append(nucleus_id)
    return initial_results, valid_nuclei_labels

In [ ]:
def visualize_segmentation(purple_channel, lamin_rings, overlay, labeled_nuclei, valid_nuclei_labels, file_name, title, save_path=None):
    if any(x is None for x in [purple_channel, lamin_rings, overlay, labeled_nuclei]):
        return
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(purple_channel, cmap='gray')
    plt.title(f'Purple Channel (Lamin) - {file_name}')
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(lamin_rings, cmap='gray')
    plt.title(f'Purple Lamin Rings - {file_name}')
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    for region in regionprops(labeled_nuclei):
        if region.label in valid_nuclei_labels:
            y, x = region.centroid
            plt.text(x, y, str(region.label), color='white', fontsize=8, ha='center', va='center')
    plt.title(title)
    plt.axis('off')
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close()
    else:
        plt.show()

In [ ]:
# Segment 6: Interactive Nucleus Selection
def interactive_selection(purple_channel, labeled_nuclei, valid_nuclei_labels, file_name):
    if any(x is None for x in [purple_channel, labeled_nuclei, valid_nuclei_labels]):
        print("Error: Invalid data for interactive selection.")
        return []
    print(f"\nValid nucleus IDs for {file_name}: {valid_nuclei_labels}")
    print("Left-click to exclude nuclei, right-click to include back. Close window to finish.")

    def create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels, exclude_ids):
        overlay = np.stack([purple_channel, purple_channel, purple_channel], axis=-1)
        overlay_max = overlay.max()
        if overlay_max == 0:
            overlay = np.zeros_like(overlay, dtype=np.uint8) + 128  # Default gray
        else:
            overlay = (overlay / overlay_max * 255).astype(np.uint8)
        valid_mask = np.isin(labeled_nuclei, valid_nuclei_labels)
        exclude_mask = np.isin(labeled_nuclei, exclude_ids)
        overlay[valid_mask & ~exclude_mask, 0] = 255  # Yellow for included
        overlay[valid_mask & ~exclude_mask, 1] = 255
        overlay[valid_mask & ~exclude_mask, 2] = 0
        overlay[exclude_mask, 0] = 255  # Red for excluded
        overlay[exclude_mask, 1] = 0
        overlay[exclude_mask, 2] = 0
        alpha = 0.3
        overlay = (alpha * overlay + (1 - alpha) * np.stack([purple_channel, purple_channel, purple_channel], axis=-1)).astype(np.uint8)
        return overlay

    fig, ax = plt.subplots(figsize=(12, 12))
    overlay = create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels, [])
    ax.imshow(overlay)
    for region in regionprops(labeled_nuclei):
        if region.label in valid_nuclei_labels:
            y, x = region.centroid
            ax.text(x, y, str(region.label), color='white', fontsize=8, ha='center', va='center')
    ax.set_title(f'Left-click to exclude, right-click to include - {file_name}')
    ax.axis('off')

    exclude_ids = []
    def on_click(event):
        if event.inaxes != ax:
            return
        x, y = int(event.xdata), int(event.ydata)
        tolerance = 10
        label_at_click = 0
        min_dist = tolerance
        for region in regionprops(labeled_nuclei):
            if region.label not in valid_nuclei_labels:
                continue
            ry, rx = region.centroid
            dist = np.sqrt((y - ry)**2 + (x - rx)**2)
            if dist < min_dist:
                min_dist = dist
                label_at_click = region.label

        if label_at_click in valid_nuclei_labels:
            if event.button == 1:  # Left-click to exclude
                if label_at_click not in exclude_ids:
                    exclude_ids.append(label_at_click)
                    print(f"Excluding nucleus ID: {label_at_click}")
            elif event.button == 3:  # Right-click to include
                if label_at_click in exclude_ids:
                    exclude_ids.remove(label_at_click)
                    print(f"Including nucleus ID: {label_at_click}")

            ax.clear()
            overlay = create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels, exclude_ids)
            ax.imshow(overlay)
            for region in regionprops(labeled_nuclei):
                if region.label in valid_nuclei_labels:
                    y, x = region.centroid
                    ax.text(x, y, str(region.label), color='white', fontsize=8, ha='center', va='center')
            ax.set_title(f'Left-click to exclude, right-click to include - {file_name}')
            ax.axis('off')
            plt.draw()

    fig.canvas.mpl_connect('button_press_event', on_click)
    plt.show(block=True)

    invalid_ids = [id for id in exclude_ids if id not in valid_nuclei_labels]
    if invalid_ids:
        print(f"Warning: Invalid nucleus IDs {invalid_ids} ignored.")
        exclude_ids = [id for id in exclude_ids if id in valid_nuclei_labels]

    print(f"Excluded nuclei: {exclude_ids}")
    selection_file = os.path.join(directory, f"{file_name}_excluded_ids.txt")
    with open(selection_file, 'w') as f:
        f.write(','.join(map(str, exclude_ids)))
    print(f"Excluded IDs saved to {selection_file}")
    return exclude_ids

In [ ]:
# Segment 7: Final Quantification
def final_quantification(initial_results, exclude_ids):
    results = [r for r in initial_results if r['Nucleus_ID'] not in exclude_ids]
    print(f"\nQuantified results after excluding nuclei {exclude_ids}:")
    if results:
        df = pd.DataFrame(results)
        print(df.to_string(index=False))
        return results
    else:
        print("No nuclei remain after exclusion.")
        return []

In [ ]:
# Segment 8: Update Summary
def update_summary(file_name, results):
    if results:
        avg_area = np.mean([r['Area'] for r in results])
        avg_green_intensity = np.mean([r['Mean_Green_Intensity'] for r in results])
        avg_int_den = np.mean([r['IntDen'] for r in results])
        total_nuclei = len(results)
    else:
        avg_area = 0
        avg_green_intensity = 0
        avg_int_den = 0
        total_nuclei = 0
    summary = {
        'File_Name': file_name,
        'Total_Nuclei': total_nuclei,
        'Average_Area': avg_area,
        'Average_Green_Intensity': avg_green_intensity,
        'Average_IntDen': avg_int_den
    }
    return summary

In [ ]:
# Segment 9: Visualize Updated Segmentation
def visualize_updated(purple_channel, lamin_rings, overlay, labeled_nuclei, valid_nuclei_labels, file_name):
    if any(x is None for x in [purple_channel, lamin_rings, overlay, labeled_nuclei]):
        print("Error: Invalid data for updated visualization.")
        return
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(purple_channel, cmap='gray')
    plt.title(f'Purple Channel (Lamin) - {file_name}')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(lamin_rings, cmap='gray')
    plt.title(f'Purple Lamin Rings - {file_name}')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    for region in regionprops(labeled_nuclei):
        if region.label in valid_nuclei_labels:
            y, x = region.centroid
            plt.text(x, y, str(region.label), color='white', fontsize=8, ha='center', va='center')
    plt.title(f'Retained Nuclei (Yellow, Labeled) - {file_name}')
    plt.axis('off')
    plt.show()

In [ ]:
def create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels, exclude_ids=None):
    if exclude_ids is None:
        exclude_ids = []
    overlay = np.stack([purple_channel, purple_channel, purple_channel], axis=-1)
    overlay_max = overlay.max()
    if overlay_max == 0:
        overlay = np.zeros_like(overlay, dtype=np.uint8) + 128
    else:
        overlay = (overlay / overlay_max * 255).astype(np.uint8)
    valid_mask = np.isin(labeled_nuclei, valid_nuclei_labels)
    exclude_mask = np.isin(labeled_nuclei, exclude_ids)
    overlay[valid_mask & ~exclude_mask, 0] = 255  # Yellow for included
    overlay[valid_mask & ~exclude_mask, 1] = 255
    overlay[valid_mask & ~exclude_mask, 2] = 0
    overlay[exclude_mask, 0] = 255  # Red for excluded
    overlay[exclude_mask, 1] = 0
    overlay[exclude_mask, 2] = 0
    alpha = 0.3
    overlay = (alpha * overlay + (1 - alpha) * np.stack([purple_channel, purple_channel, purple_channel], axis=-1)).astype(np.uint8)
    return overlay

def process_file(file_path, file_name):
    image = load_image(file_path)
    if image is None:
        return [], None, None, None, None
    purple_channel, green_channel = separate_channels(image)
    if purple_channel is None or green_channel is None:
        return [], None, None, None, None
    lamin_rings, filled_nuclei, labeled_nuclei = segment_nuclei(purple_channel)
    if labeled_nuclei is None:
        return [], None, None, None, None
    initial_results, valid_nuclei_labels = quantify_nuclei(labeled_nuclei, green_channel)
    overlay = create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels)
    return initial_results, purple_channel, lamin_rings, labeled_nuclei, valid_nuclei_labels, overlay

In [ ]:
def segment_nuclei(purple_channel, sigma=2, thresh_factor_low=0.7, thresh_factor_high=1.3, min_distance=10, min_nucleus_area=2000):
    if purple_channel is None:
        print("Error: No purple channel data for segmentation.")
        return None, None, None
    
    smoothed = gaussian(purple_channel, sigma=sigma)
    base_thresh = threshold_otsu(smoothed)
    lamin_rings = smoothed > (base_thresh * thresh_factor_low)
    lamin_rings = binary_opening(lamin_rings, disk(3))
    filled_nuclei = binary_fill_holes(lamin_rings)
    distance = distance_transform_edt(filled_nuclei)
    
    if distance.max() == 0:
        print("Error: Distance transform is empty.")
        return None, None, None
    
    seeds = peak_local_max(distance, min_distance=min_distance, labels=filled_nuclei)
    if len(seeds) == 0:
        print("Error: No seeds detected.")
        return None, None, None
    
    lamin_seeds = np.zeros_like(filled_nuclei, dtype=np.int32)
    for idx, (y, x) in enumerate(seeds):
        lamin_seeds[y, x] = idx + 1
    
    labeled_nuclei = watershed(-distance, lamin_seeds, mask=filled_nuclei)
    for region in regionprops(labeled_nuclei):
        if region.area < min_nucleus_area:
            labeled_nuclei[labeled_nuclei == region.label] = 0
    
    labeled_nuclei = label(labeled_nuclei)
    print(f"Final number of labeled nuclei: {len(np.unique(labeled_nuclei)) - 1}")
    return lamin_rings, filled_nuclei, labeled_nuclei

In [ ]:

for file_name in lsm_files:
    file_path = os.path.join(directory, file_name)
    print(f"\nProcessing file: {file_name}")
    initial_results, purple_channel, lamin_rings, labeled_nuclei, valid_nuclei_labels, overlay = process_file(file_path, file_name)
    if not initial_results:
        continue
    visualize_segmentation(purple_channel, lamin_rings, overlay, labeled_nuclei, valid_nuclei_labels, file_name, title=f"Initial Segmentation - {file_name}")
    exclude_ids = interactive_selection(purple_channel, labeled_nuclei, valid_nuclei_labels, file_name)
    results = final_quantification(initial_results, exclude_ids)
    all_results.extend(results)
    summary = update_summary(file_name, results)
    all_files_summary.append(summary)
    updated_overlay = create_overlay(purple_channel, labeled_nuclei, [id for id in valid_nuclei_labels if id not in exclude_ids])
    visualize_segmentation(purple_channel, lamin_rings, updated_overlay, labeled_nuclei, valid_nuclei_labels, file_name, title=f"Updated Segmentation - {file_name}")

In [ ]:
# Step 9: Save all results to a single CSV
if all_results:
    results_df = pd.DataFrame(all_results)
    output_file = os.path.join(directory, "all_quantification.csv")
    results_df.to_csv(output_file, index=False)
    print(f"\nAll quantification results saved to {output_file}")
else:
    print("\nNo quantification results to save.")

# Step 10: Display and save summary
print("\nSummary across all files:")
summary_df = pd.DataFrame(all_files_summary)
print(summary_df.to_string(index=False))
summary_file = os.path.join(directory, "summary_quantification.csv")
summary_df.to_csv(summary_file, index=False)
print(f"\nSummary saved to {summary_file}")
print("\nAll files processed successfully!")

In [ ]:
def separate_channels(image):
    if image is None or image.ndim != 3:
        print("Error: Invalid image data for channel separation.")
        return None, None
    purple_channel = image[2]
    green_channel = image[0]
    print("Purple channel shape:", purple_channel.shape)
    print("Purple channel min/max:", purple_channel.min(), purple_channel.max())
    print("Green channel shape:", green_channel.shape)
    print("Green channel min/max:", green_channel.min(), green_channel.max())
    return purple_channel, green_channel

def segment_nuclei(purple_channel, sigma=2, thresh_factor_low=0.7, thresh_factor_high=1.3, min_distance=10, min_nucleus_area=2000):
    if purple_channel is None:
        print("Error: No purple channel data for segmentation.")
        return None, None, None
    smoothed = gaussian(purple_channel, sigma=sigma)
    base_thresh = threshold_otsu(smoothed)
    lamin_rings = smoothed > (base_thresh * thresh_factor_low)
    lamin_rings = binary_opening(lamin_rings, disk(3))
    filled_nuclei = binary_fill_holes(lamin_rings)
    distance = distance_transform_edt(filled_nuclei)
    if distance.max() == 0:
        print("Error: Distance transform is empty.")
        return None, None, None
    seeds = peak_local_max(distance, min_distance=min_distance, labels=filled_nuclei)
    if len(seeds) == 0:
        print("Error: No seeds detected.")
        return None, None, None
    lamin_seeds = np.zeros_like(filled_nuclei, dtype=np.int32)
    for idx, (y, x) in enumerate(seeds):
        lamin_seeds[y, x] = idx + 1
    labeled_nuclei = watershed(-distance, lamin_seeds, mask=filled_nuclei)
    for region in regionprops(labeled_nuclei):
        if region.area < min_nucleus_area:
            labeled_nuclei[labeled_nuclei == region.label] = 0
    labeled_nuclei = label(labeled_nuclei)
    print(f"Final number of labeled nuclei: {len(np.unique(labeled_nuclei)) - 1}")
    return lamin_rings, filled_nuclei, labeled_nuclei

def quantify_nuclei(labeled_nuclei, green_channel):
    if labeled_nuclei is None or green_channel is None:
        print("Error: Invalid data for quantification.")
        return [], []
    initial_results = []
    min_nucleus_area = 2000
    max_nucleus_area = 10000
    valid_nuclei_labels = []
    for region in regionprops(labeled_nuclei, intensity_image=green_channel):
        if min_nucleus_area <= region.area <= max_nucleus_area:
            nucleus_id = region.label
            area_pixels = region.area
            area_um2 = area_pixels * AREA_UM2_PER_PIXEL2
            mean_intensity = region.mean_intensity
            raw_intden = np.sum(region.intensity_image[region.image])
            initial_results.append({
                'Nucleus_ID': nucleus_id,
                'Area_pixels': area_pixels,
                'Area_um2': area_um2,
                'Mean_Intensity': mean_intensity,
                'RawIntDen': raw_intden
            })
            valid_nuclei_labels.append(nucleus_id)
    return initial_results, valid_nuclei_labels

def visualize_segmentation(purple_channel, lamin_rings, overlay, labeled_nuclei, valid_nuclei_labels, file_name, title, save_path=None):
    if any(x is None for x in [purple_channel, lamin_rings, overlay, labeled_nuclei]):
        return
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(purple_channel, cmap='gray')
    plt.title(f'Purple Channel (Lamin) - {file_name}')
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(lamin_rings, cmap='gray')
    plt.title(f'Purple Lamin Rings - {file_name}')
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    for region in regionprops(labeled_nuclei):
        if region.label in valid_nuclei_labels:
            y, x = region.centroid
            plt.text(x, y, str(region.label), color='white', fontsize=8, ha='center', va='center')
    plt.title(title)
    plt.axis('off')
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close()
    else:
        plt.show()

def interactive_selection(purple_channel, labeled_nuclei, valid_nuclei_labels, file_name):
    if any(x is None for x in [purple_channel, labeled_nuclei, valid_nuclei_labels]):
        print("Error: Invalid data for interactive selection.")
        return []
    print(f"\nValid nucleus IDs for {file_name}: {valid_nuclei_labels}")
    print("Left-click to exclude nuclei, right-click to include back. Close window to finish.")

    def create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels, exclude_ids):
        overlay = np.stack([purple_channel, purple_channel, purple_channel], axis=-1)
        overlay_max = overlay.max()
        if overlay_max == 0:
            overlay = np.zeros_like(overlay, dtype=np.uint8) + 128
        else:
            overlay = (overlay / overlay_max * 255).astype(np.uint8)
        valid_mask = np.isin(labeled_nuclei, valid_nuclei_labels)
        exclude_mask = np.isin(labeled_nuclei, exclude_ids)
        overlay[valid_mask & ~exclude_mask, 0] = 255
        overlay[valid_mask & ~exclude_mask, 1] = 255
        overlay[valid_mask & ~exclude_mask, 2] = 0
        overlay[exclude_mask, 0] = 255
        overlay[exclude_mask, 1] = 0
        overlay[exclude_mask, 2] = 0
        alpha = 0.3
        overlay = (alpha * overlay + (1 - alpha) * np.stack([purple_channel, purple_channel, purple_channel], axis=-1)).astype(np.uint8)
        return overlay

    fig, ax = plt.subplots(figsize=(12, 12))
    overlay = create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels, [])
    ax.imshow(overlay)
    for region in regionprops(labeled_nuclei):
        if region.label in valid_nuclei_labels:
            y, x = region.centroid
            ax.text(x, y, str(region.label), color='white', fontsize=8, ha='center', va='center')
    ax.set_title(f'Left-click to exclude, right-click to include - {file_name}')
    ax.axis('off')

    exclude_ids = []
    def on_click(event):
        if event.inaxes != ax:
            return
        x, y = int(event.xdata), int(event.ydata)
        tolerance = 10
        label_at_click = 0
        min_dist = tolerance
        for region in regionprops(labeled_nuclei):
            if region.label not in valid_nuclei_labels:
                continue
            ry, rx = region.centroid
            dist = np.sqrt((y - ry)**2 + (x - rx)**2)
            if dist < min_dist:
                min_dist = dist
                label_at_click = region.label
        if label_at_click in valid_nuclei_labels:
            if event.button == 1:
                if label_at_click not in exclude_ids:
                    exclude_ids.append(label_at_click)
                    print(f"Excluding nucleus ID: {label_at_click}")
            elif event.button == 3:
                if label_at_click in exclude_ids:
                    exclude_ids.remove(label_at_click)
                    print(f"Including nucleus ID: {label_at_click}")
            ax.clear()
            overlay = create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels, exclude_ids)
            ax.imshow(overlay)
            for region in regionprops(labeled_nuclei):
                if region.label in valid_nuclei_labels:
                    y, x = region.centroid
                    ax.text(x, y, str(region.label), color='white', fontsize=8, ha='center', va='center')
            ax.set_title(f'Left-click to exclude, right-click to include - {file_name}')
            ax.axis('off')
            plt.draw()

    fig.canvas.mpl_connect('button_press_event', on_click)
    plt.show(block=True)

    invalid_ids = [id for id in exclude_ids if id not in valid_nuclei_labels]
    if invalid_ids:
        print(f"Warning: Invalid nucleus IDs {invalid_ids} ignored.")
        exclude_ids = [id for id in exclude_ids if id in valid_nuclei_labels]
    print(f"Excluded nuclei: {exclude_ids}")
    selection_file = os.path.join(directory, f"{file_name}_excluded_ids.txt")
    with open(selection_file, 'w') as f:
        f.write(','.join(map(str, exclude_ids)))
    print(f"Excluded IDs saved to {selection_file}")
    return exclude_ids

def final_quantification(initial_results, exclude_ids):
    results = [r for r in initial_results if r['Nucleus_ID'] not in exclude_ids]
    print(f"\nQuantified results after excluding nuclei {exclude_ids}:")
    if results:
        df = pd.DataFrame(results)
        print(df.to_string(index=False))
        return results
    else:
        print("No nuclei remain after exclusion.")
        return []

def update_summary(file_name, results):
    if results:
        avg_area_pixels = np.mean([r['Area_pixels'] for r in results])
        avg_area_um2 = np.mean([r['Area_um2'] for r in results])
        avg_mean_intensity = np.mean([r['Mean_Intensity'] for r in results])
        avg_raw_intden = np.mean([r['RawIntDen'] for r in results])
        total_nuclei = len(results)
    else:
        avg_area_pixels = 0
        avg_area_um2 = 0
        avg_mean_intensity = 0
        avg_raw_intden = 0
        total_nuclei = 0
    summary = {
        'File_Name': file_name,
        'Total_Nuclei': total_nuclei,
        'Average_Area_pixels': avg_area_pixels,
        'Average_Area_um2': avg_area_um2,
        'Average_Mean_Intensity': avg_mean_intensity,
        'Average_RawIntDen': avg_raw_intden
    }
    return summary

def visualize_updated(purple_channel, lamin_rings, overlay, labeled_nuclei, valid_nuclei_labels, file_name):
    if any(x is None for x in [purple_channel, lamin_rings, overlay, labeled_nuclei]):
        print("Error: Invalid data for updated visualization.")
        return
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(purple_channel, cmap='gray')
    plt.title(f'Purple Channel (Lamin) - {file_name}')
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(lamin_rings, cmap='gray')
    plt.title(f'Purple Lamin Rings - {file_name}')
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    for region in regionprops(labeled_nuclei):
        if region.label in valid_nuclei_labels:
            y, x = region.centroid
            plt.text(x, y, str(region.label), color='white', fontsize=8, ha='center', va='center')
    plt.title(f'Retained Nuclei (Yellow, Labeled) - {file_name}')
    plt.axis('off')
    plt.show()

def create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels, exclude_ids=None):
    if exclude_ids is None:
        exclude_ids = []
    overlay = np.stack([purple_channel, purple_channel, purple_channel], axis=-1)
    overlay_max = overlay.max()
    if overlay_max == 0:
        overlay = np.zeros_like(overlay, dtype=np.uint8) + 128
    else:
        overlay = (overlay / overlay_max * 255).astype(np.uint8)
    valid_mask = np.isin(labeled_nuclei, valid_nuclei_labels)
    exclude_mask = np.isin(labeled_nuclei, exclude_ids)
    overlay[valid_mask & ~exclude_mask, 0] = 255
    overlay[valid_mask & ~exclude_mask, 1] = 255
    overlay[valid_mask & ~exclude_mask, 2] = 0
    overlay[exclude_mask, 0] = 255
    overlay[exclude_mask, 1] = 0
    overlay[exclude_mask, 2] = 0
    alpha = 0.3
    overlay = (alpha * overlay + (1 - alpha) * np.stack([purple_channel, purple_channel, purple_channel], axis=-1)).astype(np.uint8)
    return overlay

def process_file(file_path, file_name):
    image = load_image(file_path)
    if image is None:
        return [], None, None, None, None, None
    purple_channel, green_channel = separate_channels(image)
    if purple_channel is None or green_channel is None:
        return [], None, None, None, None, None
    lamin_rings, filled_nuclei, labeled_nuclei = segment_nuclei(purple_channel)
    if labeled_nuclei is None:
        return [], None, None, None, None, None
    initial_results, valid_nuclei_labels = quantify_nuclei(labeled_nuclei, green_channel)
    overlay = create_overlay(purple_channel, labeled_nuclei, valid_nuclei_labels)
    return initial_results, purple_channel, lamin_rings, labeled_nuclei, valid_nuclei_labels, overlay

for file_name in lsm_files:
    file_path = os.path.join(directory, file_name)
    print(f"\nProcessing file: {file_name}")
    initial_results, purple_channel, lamin_rings, labeled_nuclei, valid_nuclei_labels, overlay = process_file(file_path, file_name)
    if not initial_results:
        continue
    visualize_segmentation(purple_channel, lamin_rings, overlay, labeled_nuclei, valid_nuclei_labels, file_name, title=f"Initial Segmentation - {file_name}")
    exclude_ids = interactive_selection(purple_channel, labeled_nuclei, valid_nuclei_labels, file_name)
    results = final_quantification(initial_results, exclude_ids)
    all_results.extend(results)
    summary = update_summary(file_name, results)
    all_files_summary.append(summary)
    updated_overlay = create_overlay(purple_channel, labeled_nuclei, [id for id in valid_nuclei_labels if id not in exclude_ids])
    visualize_segmentation(purple_channel, lamin_rings, updated_overlay, labeled_nuclei, valid_nuclei_labels, file_name, title=f"Updated Segmentation - {file_name}")

if all_results:
    results_df = pd.DataFrame(all_results)
    output_file = os.path.join(directory, "all_quantification.csv")
    results_df.to_csv(output_file, index=False)
    print(f"\nAll quantification results saved to {output_file}")
else:
    print("\nNo quantification results to save.")

print("\nSummary across all files:")
summary_df = pd.DataFrame(all_files_summary)
print(summary_df.to_string(index=False))
summary_file = os.path.join(directory, "summary_quantification.csv")
summary_df.to_csv(summary_file, index=False)
print(f"\nSummary saved to {summary_file}")
print("\nAll files processed successfully!")